# Implementation of 9-layer CNN (pytorch)

In [18]:
# General imports 
import sys
import os 
sys.path.insert(1, os.path.join(os.pardir, 'src'))
from itertools import product

# Data imports
import torch
import plotly.express as px
import numpy as np
import mlflow
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

# Homebrew imports 
import model
from utils import one_hot_encode_index
from optimizers import Adam
from activations import Softmax, ReLU
from layers import Dropout, LinearLayer, ConvolutionLayer, PoolingLayer, FlattenLayer
from loss import CategoricalCrossEntropyLoss

## TESTING 
import importlib
importlib.reload(model)
##

<module 'model' from '..\\src\\model.py'>

## Data loaders

In [19]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(32),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(33),
                                      transforms.CenterCrop(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                    ])

# setting up data loaders
data_dir = os.path.join(os.pardir, 'data', 'Plant_leave_diseases_32')

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'validation'), transform=test_transforms)

### Train config

In [20]:
# Configs 
config = {
    'max_epochs': 100,
    'learning_rate': 0.003,
    'resolution': 32,
    'name': 'CNN_Basic_1CNN_module_homebrew'
}

## Model

In [21]:
mdl = model.Model(Adam(learning_rate=config['learning_rate']),
                      CategoricalCrossEntropyLoss())

# Config early stop 
mdl.add_early_stop(5)

mdl.set_save_config(model_name=config['name'], save_path=os.path.join(os.pardir, 'model'))

# Defining architecture 

mdl.set_sequence([
                    ConvolutionLayer(3, 32, 3),
                    ReLU(),
                    PoolingLayer(32, 2),
                    FlattenLayer(),
                    LinearLayer(7200, 1568),
                    ReLU(),
                    LinearLayer(1568, 128),
                    ReLU(),
                    LinearLayer(128, 39),
                    Softmax()
                ])
print(mdl)

Model Architecture: 
	 (0): ConvolutionLayer (Trainable: False)
	 (1): ReLU (Trainable: False)
	 (2): PoolingLayer (Trainable: False)
	 (3): FlattenLayer (Trainable: False)
	 (4): LinearLayer (Trainable: True)
	 (5): ReLU (Trainable: False)
	 (6): LinearLayer (Trainable: True)
	 (7): ReLU (Trainable: False)
	 (8): LinearLayer (Trainable: True)
	 (9): Softmax (Trainable: False)



## Training 

In [22]:
mlflow.set_experiment("Plant Leaf Disease")

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)


with mlflow.start_run():
    mlflow.log_param('framework', 'homebrew')
    mlflow.log_param('data_split', '90/10')
    mlflow.log_param('type', 'Basic_CNN')
    mlflow.log_params(config)
    mdl.train_with_loader(train_loader, epochs=config['max_epochs'], validation_loader=validation_loader, cls_count=39, log_freq=5)

=== Epoch: 1 ===
Step: 0/865, accuracy0.047, loss3.687, learning rate 0.0030000 
Step: 5/865, accuracy0.062, loss5.572, learning rate 0.0030000 
Step: 10/865, accuracy0.094, loss6.248, learning rate 0.0030000 
Step: 15/865, accuracy0.078, loss5.391, learning rate 0.0030000 
Step: 20/865, accuracy0.094, loss4.016, learning rate 0.0030000 
Step: 25/865, accuracy0.094, loss4.448, learning rate 0.0030000 
Step: 30/865, accuracy0.031, loss3.667, learning rate 0.0030000 
Step: 35/865, accuracy0.156, loss3.736, learning rate 0.0030000 
Step: 40/865, accuracy0.094, loss3.625, learning rate 0.0030000 
Step: 45/865, accuracy0.141, loss3.555, learning rate 0.0030000 
Step: 50/865, accuracy0.031, loss3.659, learning rate 0.0030000 
Step: 55/865, accuracy0.141, loss3.456, learning rate 0.0030000 
Step: 60/865, accuracy0.172, loss3.501, learning rate 0.0030000 
Step: 65/865, accuracy0.125, loss3.312, learning rate 0.0030000 
Step: 70/865, accuracy0.109, loss3.403, learning rate 0.0030000 
Step: 75/8

In [ ]:
mdl.layers[3].output.shape

(64, 7200)